## Filtering
The filtering was done using a Kalman Filter. We designed it to use the position and the angle given by the camera as measurement and the speed of the robot, given by the sensors of the wheels, as prediction. For simplicity of use, we decided to create a Kalman_Filter class, so we would have less arguments to give to each functions. However, this class needs to have access to the class Vision and to be given the speed of the robot to work properly.  


### Equations
The Kalman filter merge together a prediction of the future state of the system with a measure of these states to be more precise. The prediction is govern by the system equations. In this section, we are going to describe the equations used in the model and the ones used to implement the merging part of the filter. 
First, we need to convert the informations we get from the sensors to something we can use:
$v = (v_r + v_l)/2$. This is the translational speed and $w = (v_r - v_l)/b$, which is the rotationnal speed. $v_r$ is the speed of the right wheel, $v_l$ the left one and $b$ is the width of the robot, wheel to wheel. From $w$, we can compute the variation of angle that the robot does when it moves: $\alpha = w * \delta t$.
We have three states for our filter, which are $x$, $y$, and $\theta$, respectively the position on the x and y axis and the orientation from the x axis. The state vector is called $\rho$ and the vector of input ($[v ; w]$) is called u for simplication of writing. Therefore, the model equation is the following: $\rho_{k+1} = A * \rho_k + B * u_k$. We now need to determine the matrixes A and B. For this, we have to know how the system evolve with each steps. For x, we can find that $x_{k+1} = x_k + v * \cos(\alpha + \theta) * \delta t$ and for y: $y_{k+1} = y_k + v *\sin(\alpha + \theta)* \delta t$. Finally, for $\theta$, we have $\theta_{k+1} = \theta_k + w *\delta t$. From this we can find that the matrix A is  \begin{bmatrix} 1 & 0 & 0\\ 0 & 1 & 0 \\ 0 & 0 & 1\end{bmatrix} 